In [1]:
from finlab import data

#guide can be found here
#https://mrjbq7.github.io/ta-lib/funcs.html
#https://twopirllc.github.io/pandas-ta/#indicators-by-category
#pip install --upgrade numpy
#!rew install ta-lib
#pip install pandas-lib
from finlab import data
import finlab
from finlab.backtest import sim
from finlab import backtest
finlab.login(api_token="QNBAVxVcsFjg0vZ5Um8P4XVtRWnhnMtznBpGjGHXqQ+vBYWkN6kXnXDrW70KhH4v#free")


INFO:numexpr.utils:NumExpr defaulting to 4 threads.


輸入成功!


In [5]:
#策略開發10分鐘上手
from finlab import data
from finlab.backtest import sim
close = data.get('price:收盤價')
pb = data.get('price_earning_ratio:股價淨值比')  
sma20 = close.average(20)
sma60 = close.average(60)
entries = close > sma20
exits = close < sma60
position = entries.hold_until(exits, rank=-pb)
backtest_report = sim(position,resample="Q")

In [ ]:
# 收盤價剛站上布林通道上緣
df=data.get('price:收盤價')
mean_20=df.rolling(20).mean()
std_value=df.rolling(20).std()
up=mean_20+std_value*2
cond1=(df>up)&(df.shift()<up)
cond2=(df>5)&(df<200)
position=(cond1&cond2)
position=position[position>0]
position=position.is_smallest(20)

In [ ]:
#均線多頭
df=data.get('price:收盤價')
df=df[(df.index>'2015')&(df.index<'2019')]
cond1=df>df.rolling(5).mean()
cond2=df>df.rolling(10,min_periods=5).mean()
cond3=df>df.rolling(20,min_periods=10).mean()
cond4=df>df.rolling(60,min_periods=40).mean()
position=(df*(cond1&cond2&cond3&cond4))
position=position[position>0]
position=position.is_largest(10)

In [8]:
#股價創新高動能
from finlab.backtest import sim
close = data.get("price:收盤價")
roe = data.get("fundamental_features:ROE稅後")

position = (close == close.rolling(250).max())&roe>0

report = sim(position, resample="M", name="創年新高策略")
report.display()

INFO:finlab.data:company_basic_info -- Daily data usage: 541.5 / 5000 MB
INFO:finlab.data:financial_statements_upload_detail:upload_date -- Daily data usage: 543.1 / 5000 MB


1446    0.045455
1789    0.045455
2367    0.045455
2530    0.045455
3163    0.045455
3289    0.045455
3664    0.045455
4162    0.045455
4168    0.045455
4506    0.045455
4763    0.045455
5225    0.045455
6143    0.045455
6153    0.045455
8084    0.045455
8109    0.045455
8403    0.045455
8420    0.045455
8916    0.045455
8933    0.045455
8938    0.045455
9942    0.045455
Name: 2022-09-30 00:00:00, dtype: float64

Timestamp('2022-09-30 00:00:00')

In [6]:
#高 RSI 技術指標策略
from finlab import data
from finlab.backtest import sim
#foreign_buy=data.get('institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)')
#foreign_cont=foreign_buy>0
#increase=data.get('internal_equity_changes:董監增加股數')
#cond7=increase>0
roe = data.get("fundamental_features:ROE稅後")
# 選出 RSI 最大的 20 檔股票
rsi = data.indicator('rsi')
position = rsi.is_largest(40)
# 回測，每週(W)調整一次
report = sim(position, resample='W',, name="高RSI策略")

INFO:finlab.data:fundamental_features:ROE稅後 -- Daily data usage: 447.2 / 5000 MB
INFO:finlab.data:price:開盤價 -- Daily data usage: 468.5 / 5000 MB
INFO:finlab.data:price:最高價 -- Daily data usage: 489.8 / 5000 MB
INFO:finlab.data:price:最低價 -- Daily data usage: 511.0 / 5000 MB
INFO:finlab.data:price:成交股數 -- Daily data usage: 540.6 / 5000 MB


In [9]:
#乖離率和財報濾網
from finlab import data
from finlab.backtest import sim
# 下載 ROE 跟收盤價
roe = data.get("fundamental_features:ROE稅後")
close = data.get("price:收盤價")
position = ((close / close.shift(60)).is_largest(30) & (roe > 0))
# 回測，每月(M)調整一次
report = sim(position, resample="M", upload=False)
report.display()
from finlab import data
from finlab.backtest import sim
# 下載 ROE 跟收盤價
roe = data.get("fundamental_features:ROE稅後")
close = data.get('price:收盤價')
position = (
      (close / close.shift(60)).is_largest(30) # 選出乖離率前 30 名的股票
    & (roe > 0) # 選出 ROE 大於 0 的股票
)

# 回測，每月(M)調整一次
report = sim(position, resample='M', name="乖離率和ROE濾網策略")

1760    0.045455
2230    0.045455
2305    0.045455
2364    0.045455
2468    0.045455
3022    0.045455
3043    0.045455
3432    0.045455
3504    0.045455
4162    0.045455
4413    0.045455
4432    0.045455
4557    0.045455
4743    0.045455
4979    0.045455
5205    0.045455
5353    0.045455
6118    0.045455
6133    0.045455
6472    0.045455
6649    0.045455
6796    0.045455
Name: 2022-09-30 00:00:00, dtype: float64

Timestamp('2022-09-30 00:00:00')

In [ ]:
#本益成長比策略
from finlab import data
from finlab.backtest import sim
rev = data.get('monthly_revenue:當月營收')
rev_ma3 = rev.average(3)
rev_ma12 = rev.average(12)
# 營收趨勢多頭策略
cond1 = rev_ma3/rev_ma12 > 1.1
cond2 = rev/rev.shift(1) > 0.9
cond_all = cond1 & cond2
pe = data.get('price_earning_ratio:本益比')
營業利益成長率 = data.get('fundamental_features:營業利益成長率')
# 本益成長比
peg = (pe/營業利益成長率)
# 本益成長比和原訊號相乘，若不持有則相乘後等於0
position = peg*(cond_all)
# 原訊號為0的不要選，若沒加這行且策略只選到7檔，之後還是會選3檔訊號為0(不持有)的補足10檔，執行這行就會排除訊號為0。
position = position[position>0]
# 選股挑本益成長比前10小的
position = position.is_smallest(10)
sim(position,resample="W")

In [11]:
#小資族優等生策略
from finlab import data
from finlab.backtest import sim
股本 = data.get('financial_statement:股本')
price = data.get('price:收盤價')
市值 = 股本 * price / 10 * 1000
df1 = data.get('financial_statement:投資活動之淨現金流入_流出')
df2 = data.get('financial_statement:營業活動之淨現金流入_流出')
自由現金流 = (df1 + df2).rolling(4).mean()
稅後淨利 = data.get('fundamental_features:經常稅後淨利')
權益總計 = data.get('financial_statement:股東權益總額')
股東權益報酬率 = 稅後淨利/ 權益總計
營業利益成長率 = data.get('fundamental_features:營業利益成長率')
當月營收 = data.get('monthly_revenue:當月營收')* 1000
當季營收 = 當月營收.rolling(4).sum()
市值營收比 = 市值 / 當季營收
foreign_buy=data.get('institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)')
foreign_buy_t=foreign_buy.loc[foreign_buy.index[1:]]
foreign_buy_tm1=foreign_buy.shift(1).loc[foreign_buy.index[1:]]
foreign_buy_diff=foreign_buy_tm1-foreign_buy
cond6=foreign_buy>0
increase=data.get('internal_equity_changes:董監增加股數')
cond7=increase>0
rsi = data.indicator('rsi')
rsi_con=rsi>75
condition1 = (市值 >1e9)
condition2 = 自由現金流 > 0
condition3 = 股東權益報酬率 > 0
condition4 = 營業利益成長率 > 0
condition5 = 市值營收比 < 5
position = condition1& condition2 & condition3 & condition4 & condition5 &cond7
report = sim(position, resample='W', name="優等生策略")
report.display()

stock_id
1231    0.023256
1477    0.023256
1712    0.023256
1817    0.023256
2063    0.023256
2385    0.023256
2439    0.023256
2467    0.023256
2493    0.023256
2504    0.023256
2612    0.023256
2615    0.023256
2618    0.023256
2753    0.023256
3221    0.023256
3264    0.023256
3324    0.023256
3548    0.023256
4439    0.023256
4532    0.023256
4535    0.023256
4545    0.023256
4915    0.023256
4923    0.023256
4933    0.023256
4960    0.023256
5209    0.023256
5258    0.023256
5493    0.023256
5520    0.023256
5533    0.023256
6121    0.023256
6167    0.023256
6208    0.023256
6239    0.023256
6242    0.023256
6270    0.023256
6570    0.023256
8070    0.023256
8088    0.023256
8103    0.023256
8401    0.023256
9962    0.023256
Name: 2022-09-04 00:00:00, dtype: float64

Timestamp('2022-09-04 00:00:00')

In [ ]:
#多空對沖
from finlab import data
from finlab import backtest
close = data.get('price:收盤價')
position = close < 0
position['2330'] = 0.5
position['1101'] = -0.5
report = backtest.sim(position, upload=False)
report.display()
from finlab import data
from finlab import backtest
pb = data.get('price_earning_ratio:股價淨值比')
# 股價淨值比前20%低
position = pb < pb.quantile_row(0.2)
position /= position.sum(axis=1)
# 多單總部位70%
position *= 0.7
# 空單總部位30%
position['0050'] = -0.3
report=backtest.sim( position, upload=False, resample='Q')
report.display()

In [ ]:
#營建產業限定策略
from finlab import data
from finlab.backtest import sim
import pandas as pd
# 限定回測產業範圍
with data.universe(category='建材營造'):
    # 資料引入
    close = data.get('price:收盤價')
    contract_debt = data.get('financial_statement:合約負債_流動')
    contract_debt_gr = (contract_debt/(contract_debt.shift())-1)
    equity = data.get('financial_statement:股本')
    # 計算合約負債佔股本的比率，比率越高代表未來潛在貢獻每股營收越高
    ce_ratio = (contract_debt / equity)
    # 選股條件
    cond1 = ce_ratio > 0.5
    cond2 = (contract_debt_gr > 0.05) & (contract_debt_gr<0.5)
    cond_all = cond1 & cond2
    result = ce_ratio * (cond_all)
    position = result[result > 0].is_largest(5)
    # position_limit 限定單檔標的最多持有30%部位，避免標的數過少時產生重壓的非系統性風險
    report = sim(position,resample='M',position_limit=0.3,stop_loss=0.1,fee_ratio=1.425/1000*0.3,name="營建股合約負債策略",upload=False)

report.display()

In [ ]:
#營收動能
from finlab import data
from finlab.backtest import sim
import pandas as pd
rev = data.get("monthly_revenue:當月營收")
rev_rf = data.get("monthly_revenue:去年同月增減(%)")
vol = data.get("price:成交股數")/1000
rev_recent_3 = rev.rolling(3).sum()
vol_avg = vol.average(10)
cond1 = (rev_recent_3/rev_recent_3.rolling(24, min_periods=12).max())==1
# 成交量條件，排除流動性過小
cond2 = vol_avg > 300
# 過濾全額交割股
full_cash_delivery_stock_filter = data.get("etl:full_cash_delivery_stock_filter")
cond_all = cond1 & cond2 & full_cash_delivery_stock_filter 
result = rev_rf*(cond_all)
position = result[result>0].is_largest(10).reindex(rev.index_str_to_date().index, method="ffill")
report = sim(position=position,stop_loss=0.3,position_limit=0.1)
report.display()

In [ ]:
#現金流量策略
from finlab import data
from finlab.backtest import sim
營業現金流 = data.get("financial_statement:營業活動之淨現金流入_流出").deadline()
投資現金流 = data.get("financial_statement:投資活動之淨現金流入_流出").deadline()
融資現金流 = data.get("financial_statement:籌資活動之淨現金流入_流出").deadline()
roe = data.get("fundamental_features:ROE稅後").deadline()
close = data.get("price:收盤價")
position = (營業現金流 > 0) & (投資現金流 > 0) & (融資現金流 < 0) & (roe > 5) & (close > 10)
from finlab import data
from finlab.backtest import sim
營業現金流 = data.get("financial_statement:營業活動之淨現金流入_流出").deadline()
投資現金流 = data.get("financial_statement:投資活動之淨現金流入_流出").deadline()
融資現金流 = data.get("financial_statement:籌資活動之淨現金流入_流出").deadline()
roe = data.get("fundamental_features:ROE稅後").deadline()
close = data.get("price:收盤價")
position = (營業現金流 > 0) & (投資現金流 > 0) & (融資現金流 < 0) & (roe > 0) & (close > 10)
# 過濾 KY股
sc = data.get("security_categories")
position_col = position.columns
ky_filter = position_col[~position_col.isin(list(sc[sc["name"].str.contains("KY")]["stock_id"]))]
position = position[ky_filter]
report = sim(position,resample="M",name="現金流正數")
report.display()

In [ ]:
#大盤指標濾網
from finlab import data
from finlab.backtest import sim
import numpy as np
# 市場多空排列家數
def ls_order_position(short=5,mid=10,long=30):
    close = data.get("price:收盤價")
    short_ma = close.average(short)
    mid_ma = close.average(mid)
    long_ma = close.average(long)
    long_order = (short_ma>=mid_ma) & (mid_ma>=long_ma)
    long_order = long_order.sum(1)
    short_order = (short_ma<mid_ma) & (mid_ma<long_ma)
    short_order = short_order.sum(1)
    entry = long_order > short_order
    cond = ~close.isna()
    position = cond & entry
    return position
close = data.get("price:收盤價")
sma20 = close.average(20)
rev = data.get('monthly_revenue:當月營收')
# 多頭排列
cond1 = (close > close.shift(20)) & (close > close.shift(60))
# 短期營收大於長期營收
cond2 = (rev.average(3) > rev.average(12)).sustain(3)
buy = cond1 & cond2
sell = sma20 < sma20.shift()
position = pd.DataFrame(np.nan, index=buy.index, columns=buy.columns)
position[buy] = 1
position[sell] = 0
position = position.ffill().fillna(0).astype(bool)
position = position & ls_order_position()
report = sim(position, resample="D", name="ls_order_position")
report.display()

In [ ]:
#布林通道策略
from finlab import data
from finlab.backtest import sim
close=data.get('price:收盤價')
upperband, middleband, lowerband = data.indicator('BBANDS', resample='D', nbdevup=2.5, nbdevdn=2.5, timeperiod=40)
entries = lowerband > close
exits = close > middleband
position = entries.hold_until(exits,nstocks_limit=100000,take_profit=0.1,stop_loss=0.15)['2330']
report=sim(position, upload=True, mae_mfe_window=40, name='護國神山救難隊')

In [ ]:
#小於六策略
#from finlab import data
from finlab import backtest
# 只買入價格小於 6 元的股票
close = data.get('price:收盤價')
position = close < 6
# 回測，每月底(M)重新調整股票權重
report = backtest.sim(position, resample='M', name="價格小於6的股票")


In [ ]:
#本益成本比策略
from finlab import data
from finlab.backtest import sim
pe = data.get('price_earning_ratio:本益比')
rev=data.get('monthly_revenue:當月營收')
rev_ma3=rev.average(3)
rev_ma12=rev.average(12)
營業利益成長率=data.get('fundamental_features:營業利益成長率').deadline()
peg=(pe/營業利益成長率)
cond1= rev_ma3/rev_ma12>1.1
cond2=rev/rev.shift(1)>0.9
cond_all=cond1 & cond2
result=peg*(cond_all)
position=result[result>0].is_smallest(10).reindex(rev.index_str_to_date().index, method='ffill')
sim(position=position,resample="W",fee_ratio=1.425/1000/3,stop_loss=0.1,name='本益成長比')

In [ ]:
#股價淨值比策略
from finlab import data
from finlab import backtest
收盤價 = data.get('price:收盤價')
股價淨值比 = data.get("price_earning_ratio:股價淨值比")
# 條件 1
收盤價大於五 = 收盤價 > 5
# 條件 2
季線 = 收盤價.rolling(60).mean()
收盤價在季線之上 = 收盤價 > 季線
# 條件 3
總條件 = (收盤價 * 股價淨值比)[收盤價大於五&收盤價在季線之上].is_smallest(20)
backtest.sim(總條件, resample='Q', live_performance_start='2021-01-01')

In [ ]:
# 營收趨勢多頭策略
from finlab import data
from finlab.backtest import sim
rev = data.get('monthly_revenue:當月營收')
rev_ma3 = rev.average(3)
rev_ma12 = rev.average(12)
cond1 = rev_ma3/rev_ma12 > 1.1
cond2 = rev/rev.shift(1) > 0.9
cond_all = cond1 & cond2
pe = data.get('price_earning_ratio:本益比')
營業利益成長率 = data.get('fundamental_features:營業利益成長率')
# 本益成長比
peg = (pe/營業利益成長率)
# 本益成長比和原訊號相乘，若不持有則相乘後等於0
position = peg*(cond_all)
# 原訊號為0的不要選，若沒加這行且策略只選到7檔，之後還是會選3檔訊號為0(不持有)的補足10檔，執行這行就會排除訊號為0。
position = position[position>0]
# 選股挑本益成長比前10小的
position = position.is_smallest(10)

In [ ]:
#肯特納通道突破策略
from finlab.backtest import sim
from finlab import data
data.set_universe(market='TSE_OTC')
adj_close = data.get('etl:adj_close').loc['2012':]
ema_5=data.indicator('ema', adjust_price=True, resample='D',timeperiod=5)
ema_10=data.indicator('ema', adjust_price=True, resample='D',timeperiod=10)
ema_20=data.indicator('ema', adjust_price=True, resample='D',timeperiod=20)
atr=data.indicator('atr', adjust_price=True, resample='D',timeperiod=10)
keltner_up=ema_10+2*atr
vol=data.get('price:成交股數')/1000
vol_avg=vol.average(20)
cond1 = (adj_close > keltner_up) & (adj_close.shift() < keltner_up.shift())
cond2 = (ema_5 > ema_10) & (ema_10 > ema_20)
cond3 = vol_avg > 100
entries =  cond1 & cond2 & cond3
exits = adj_close < keltner_up
position = entries.hold_until(exits,nstocks_limit=20, rank=vol_avg)
rr=sim(position,resample='W',position_limit=0.05)

In [ ]:
#護國神山救難隊
from finlab import data
from finlab.backtest import sim
close=data.get('price:收盤價')
upperband, middleband, lowerband = data.indicator('BBANDS', resample='D', nbdevup=2.5, nbdevdn=2.5,timeperiod=40)
entries = lowerband > close
exits = close > middleband
# 設定賣出條件
position = entries.hold_until(exits)['2330']
report = sim(position,upload=False,mae_mfe_window=40, name='護國神山救難隊')

In [ ]:
#優等生策略
from finlab import data
from finlab.backtest import sim

股本 = data.get('financial_statement:股本')
price = data.get('price:收盤價')
市值 = 股本 * price / 10 * 1000

df1 = data.get('financial_statement:投資活動之淨現金流入_流出')
df2 = data.get('financial_statement:營業活動之淨現金流入_流出')
自由現金流 = (df1 + df2).rolling(4).mean()

稅後淨利 = data.get('fundamental_features:經常稅後淨利')
權益總計 = data.get('financial_statement:股東權益總額')
股東權益報酬率 = 稅後淨利/ 權益總計

營業利益成長率 = data.get('fundamental_features:營業利益成長率')

當月營收 = data.get('monthly_revenue:當月營收')* 1000
當季營收 = 當月營收.rolling(4).sum()
市值營收比 = 市值 / 當季營收
condition1 = (市值 < 1e10)
condition2 = 自由現金流 > 0
condition3 = 股東權益報酬率 > 0
condition4 = 營業利益成長率 > 0
condition5 = 市值營收比 < 5
position = condition1 & condition2 & condition3 & condition4 & condition5
report = sim(position, resample='M', name="優等生策略", upload=False)
report.display()

In [ ]:
#ls_position_strategy
df=data.get('price:收盤價')
mean_20=df.rolling(20).mean()
std_value=df.rolling(20).std()
up=mean_20+std_value*2
# 收盤價剛站上布林通道上緣
cond1=(df>up)&(df.shift()<up)
cond2=(df>5)&(df<200)
position=(cond1&cond2)
position=position[position>0]
position=position.is_smallest(20)
close = data.get("price:收盤價")
sma20 = close.average(20)
rev = data.get('monthly_revenue:當月營收')
# 多頭排列
cond1 = (close > close.shift(20)) & (close > close.shift(60))
# 短期營收大於長期營收
cond2 = (rev.average(3) > rev.average(12)).sustain(3)
buy = cond1 & cond2
sell = sma20 < sma20.shift()
position = pd.DataFrame(np.nan, index=buy.index, columns=buy.columns)
position[buy] = 1
position[sell] = 0
position = position.ffill().fillna(0).astype(bool)
position = position & ls_order_position()
report = sim(position, resample="D", name="ls_order_position")
report.display()

In [15]:
#實戰應用(六)
df1=data.get('fundamental_features:ROE稅後')


In [26]:
#實戰應用7
total_value=data.get('dividend_tse:最近一次申報每股 (單位)淨值')*data.get('company_basic_info:已發行普通股數或TDR原發行股數')

INFO:finlab.data:dividend_tse:最近一次申報每股 (單位)淨值 -- Daily data usage: 574.3 / 5000 MB
ERROR:finlab.data:**Error: cannot cannot find file.


ValueError: unknown type object

In [ ]:
'''
條件一: 公司總市值 < 市場平均總市值

條件二: 過去五年稅後淨利皆為正
data.get('fundamental_features:經常稅後淨利')
條件三: 過去三年稅後淨利成長率皆 >= 15%
data.get('fundamental_features:經常稅後淨利').rise()>0.15.sustain(3) and 

條件四: 預估稅後淨利成長率 >= 15%
data.get('fundamental_features:經常稅後淨利').sustain

條件五: 近五年平均營運現金流量 > 近五年平均稅後淨利
data.get('fundamental_features:每股現金流量')*股數>
條件六: 近年營運現金流量 > 近年稅後淨利
data.get('fundamental_features:每股現金流量')
條件七: 近年營業利益率 >= 10%
data.get('fundamental_features:營業利益率')>10%
條件八: 近年可運用資本報酬率 >= 10%
data.get('fundamental_features:ROA綜合損益')
條件九: 最近一季負債/淨值比 < 50%
data.get('fundamental_features:總負債除總淨值')
條件十: 最近董監事持股比例 >= 20% 或是 最新一期董監事持股比例增加
data.get('internal_equity_changes:董監持有股數占比')
交易面條件

條件十一: 預估本益比 <= 20

條件十二: 預估本益比/預估稅後盈餘成長率比值 <= 1.2

條件十三: 超額報酬(月) > 0
https://medium.com/tej-api-%E9%87%91%E8%9E%8D%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90/%E5%AF%A6%E6%88%B0%E6%87%89%E7%94%A8-%E4%B8%83-%E5%90%89%E5%A7%86-%E5%8F%B2%E8%90%8A%E7%89%B9-jim-slater-%E7%A5%96%E9%AD%AF%E5%8E%9F%E5%89%87%E6%8A%95%E8%B3%87%E6%B3%95-5de2c3c445c5
'''